In [4]:
import pandas as pd
import re
import numpy as np

# Universal DEV data check

This file is designed to access all data sources and cross-reference to see where data is missing.

We should be able to do this locally (in order to access the Dropbox, which is inaccessible on talapas) and FTP to talapas to grab talapas stuff.

## Setup

### Talapas FTP

Let's set up the talapas FTP.

In [5]:
import pysftp

import paramiko
#not sure if this will really continue to wokr without an explicit reference to a key file but let's see!
pagent = paramiko.agent.Agent()
srv = pysftp.Connection(host="talapas-ln2.uoregon.edu", username="bsmith16", private_key=pagent.get_keys()[0])


## Setup Sharepoint authenticaiton

In [6]:
# from office365.sharepoint.client_context import ClientContext
# from office365.runtime.auth.authentication_context import AuthenticationContext
# from office365.sharepoint.files.file import File

In [7]:
# app_principal['client_secret']

In [8]:
# #site_url = "https://uoregon.sharepoint.com/sites/O365_DEV624/"
# site_url = "https://uoregon.sharepoint.com/sites/O365_DEV624/"
# library="Shared Documents"
# file_name = "DEV%20Participant%20Mastersheet.xlsx"

# #load client secret from json file
# import json
# with open('/Users/benjaminsmith/.ssh/sharepoint_client_secret.json') as f:
#     app_principal = json.load(f)
    
# print(app_principal)




In [9]:
# #site_url = "https://uoregon.sharepoint.com/:x:/r/sites/O365_DEV624/"
# #"https://uoregon.sharepoint.com/sites/O365_DEV624/" #403 access forbidden
# site_url = "https://uoregon.sharepoint.com/:x:/r/sites/O365_DEV624/"
# ctx_auth = AuthenticationContext(site_url)
# ctx_auth.acquire_token_for_app(client_id=app_principal['client_id'], client_secret=app_principal['client_secret'])

# ctx = ClientContext(site_url, ctx_auth)
# web = ctx.web
# ctx.load(web)
# ctx.execute_query()


In [10]:
# ctx = ClientContext(site_url, ctx_auth)
# web = ctx.web
# ctx.load(web)
# ctx.execute_query()



## paths

In [11]:
#paths for accessing from local machine
dicom_dir = '/gpfs/projects/lcni/dcm/sanlab/Berkman/DEV/'
redcap_csv_path= '../dicom_check/DEV-Sessions_DATA_2022-10-19_2234.csv'
behavioral_data_sst_all_csv_filepath = '~/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/sst_behavioral_data_all_20230119.csv'
behavioral_data_roc_all_csv_filepath = '~/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/roc_behavioral_data_all.csv'
behavioral_data_wtp_all_csv_filepath = '~/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/wtp_behavdesign_clean.csv'
self_report_behav_summary_data = '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/data_by_ppt.csv'
output_data_path = '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/'
subject_list_filepath = '../bidsQC/conversion/subject_list.txt'


#absolute paths for accessing directly from talapas
fmriprep_server_path = '/gpfs/projects/sanlab/shared/DEV/bids_data/derivatives/fmriprep'
bids_dir = '/gpfs/projects/sanlab/shared/DEV/bids_data'

In [12]:
canonical_session_set= ['ROC1', 'ROC2', 'ROC3', 'ROC4', 'SST1', 'WTP1', 'WTP2', 'WTP3', 'WTP4']

## Load items



## Participant mastersheet

We're going to load a copy of the participant mastersheet via excel


In [13]:
#load in the participant mastersheet excel file
participant_mastersheet_file='/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/DEV Participant Mastersheet_copy.xlsx'
participant_mastersheet = pd.read_excel(participant_mastersheet_file, sheet_name='Participants').iloc[:,0:2]

In [14]:
#we can tell the actual DEV ids to include based on whether the first "n=" column has a number in it
participants_included = participant_mastersheet[participant_mastersheet['n='].notna()]
participants_included.columns = ['mastersheet_ppt_number', 'mastersheet_dev_id']



### DICOMs

In [15]:
import re
#regex filter the directory 

dicom_dev_regex = 'DEV\d\d\d'
raw_dicom_folder_list = srv.listdir(dicom_dir)
#do matches using lowercase
raw_dicom_dev_folder_list = [x for x in raw_dicom_folder_list if re.search(dicom_dev_regex, x, re.IGNORECASE)]
non_dev_folders =  [x for x in raw_dicom_folder_list if not re.search(dicom_dev_regex, x, re.IGNORECASE)]
#extract the subject id from the folder name
dicom_sub_ids = [re.search(dicom_dev_regex, x,re.IGNORECASE).group(0) for x in raw_dicom_dev_folder_list]
dicom_dir_list_df = pd.DataFrame({'dicom_folder_name':raw_dicom_dev_folder_list, 'dicom_folder_name_sub_id':dicom_sub_ids}) #this is by folder

#let's create a dicom df that is by subject, wtih teh count of the number of folders per subject
dicom_df_by_sub = dicom_dir_list_df.groupby('dicom_folder_name_sub_id').count().reset_index()
#rename folder name to folder count
dicom_df_by_sub.rename(columns={'dicom_folder_name':'folder_count'}, inplace=True)

In [16]:
", ".join(dicom_sub_ids)

'DEV001, DEV001, DEV004, DEV004, DEV005, DEV005, DEV006, DEV007, DEV008, DEV008, DEV009, DEV009, DEV010, DEV010, DEV011, DEV011, DEV012, DEV012, DEV013, DEV013, DEV014, DEV014, DEV015, DEV015, DEV016, DEV016, DEV017, DEV017, DEV018, DEV018, DEV018, DEV019, DEV019, DEV020, DEV020, DEV021, DEV021, DEV022, DEV023, DEV023, DEV024, DEV024, DEV025, DEV025, DEV026, DEV026, DEV027, DEV027, DEV028, DEV028, DEV029, DEV029, DEV030, DEV030, DEV031, DEV031, DEV032, DEV033, DEV033, DEV034, DEV035, DEV035, DEV036, DEV036, DEV037, DEV038, DEV038, DEV039, DEV039, DEV040, DEV040, DEV041, DEV041, DEV042, DEV042, DEV043, DEV043, DEV044, DEV044, DEV045, DEV046, DEV046, DEV047, DEV047, DEV048, DEV048, DEV049, DEV049, DEV050, DEV050, DEV051, DEV051, DEV052, DEV052, DEV053, DEV053, DEV054, DEV054, DEV055, DEV055, DEV056, DEV056, DEV057, DEV057, DEV058, DEV058, DEV059, DEV059, DEV060, DEV060, DEV061, DEV061, DEV062, DEV062, DEV063, DEV063, DEV064, DEV064, DEV065, DEV065, DEV066, DEV066, DEV067, DEV067, DEV068,

In [17]:
non_dev_folders

['159_20200126_115140',
 'DEV_20171214',
 'DEV_20210608_170401',
 'DEV_20210714_173417',
 'Matlabtest_20191023_154406',
 'Phantom^of the opera_20180430_110701',
 'TEST999_20180219_170145',
 'matlabtest2_20191025_121247',
 'phantom_20181211_112538',
 'test dev_20180219_155612',
 'wwe_20180131_135555']

There are a bunch of incorrectly named DICOM files here that we'll need to go through. We arleady have a process for this in `DEV_scripts/org/dicom_check/` so need to follow that.

Maybe we can skip forward to the fMRIPrep data and register what is missing there vs. the REDCAP data.

### Sessions for DICOMs

We can't fully independently verify the DICOM subject lists, but we do have a `DEV_scripts/org/bidsQC/conversion/subject_list.txt` file that gets generated. And this file indicates which DICOMs are allocated to which session. So we can use that file to match DICOMs to sessions, and then check which session data is supposedly available for each DICOM.

In [91]:
subject_list_filepath

'../bidsQC/conversion/subject_list.txt'

In [92]:
dicom_dir_list_df

,dicom_folder_name,dicom_folder_name_sub_id
0,DEV001_20180406_170438,DEV001
1,DEV001_20180510_162136,DEV001
2,DEV004_20180416_183517,DEV004
3,DEV004_20180515_180522,DEV004
4,DEV005_20180425_185327,DEV005
...,...,...
542,DEV311_20220916_095943,DEV311
543,DEV312_20220804_101211,DEV312
544,DEV312_20220831_130818,DEV312
545,dev125_20190823_102411,dev125


In [93]:
#read csv, set the header manually
dicom_session_table = pd.read_csv(subject_list_filepath, header=None)
dicom_session_table.columns = ['dicom_index_folder_name', 'dicom_index_subject_id', 'dicom_index_wave_id']
dicom_session_table.dicom_index_wave_id=[re.search('wave(\d+)',x)[1] for x in dicom_session_table.dicom_index_wave_id]

#then, for each dicom, get the folder for each session, by merging the dicom index with the dicom session table
#I think we can only merge subject IDs to dicom folders by using the folders, not the subject ID
dicom_with_session_folders = pd.merge(dicom_dir_list_df, dicom_session_table, left_on= 'dicom_folder_name',right_on='dicom_index_folder_name', how='outer')

Part of the purpose of the DICOM analysis is that we use every DICOM folder we get. Indexing on the DICOM folder name therefore seems reasonable.

In [94]:
dicom_with_session_folders

,dicom_folder_name,dicom_folder_name_sub_id,dicom_index_folder_name,dicom_index_subject_id,dicom_index_wave_id
0,DEV001_20180406_170438,DEV001,DEV001_20180406_170438,DEV001,1
1,DEV001_20180510_162136,DEV001,DEV001_20180510_162136,DEV001,2
2,DEV004_20180416_183517,DEV004,DEV004_20180416_183517,DEV004,1
3,DEV004_20180515_180522,DEV004,DEV004_20180515_180522,DEV004,2
4,DEV005_20180425_185327,DEV005,DEV005_20180425_185327,DEV005,1
...,...,...,...,...,...
544,DEV312_20220831_130818,DEV312,DEV312_20220831_130818,DEV312,2
545,dev125_20190823_102411,dev125,dev125_20190823_102411,DEV125,2
546,dev133_20190808_171200,dev133,dev133_20190808_171200,DEV133,1
547,NaN,NaN,DEV_20210608_170401,DEV198,3


In [95]:
canonical_dicom_session_set = [
    #tasks
    'ROC1', 'ROC2', 'ROC3', 'ROC4', 'SST1', 'WTP1', 'WTP2', 'WTP3', 'WTP4', "Resting_1","Resting_2",
    #more resting
    'Resting_1_PhysioLog', 'Resting_2_PhysioLog',
    #other
    'se_epi_fieldmap_ap', 'se_epi_fieldmap_pa', 'aa_scout', 'aa_scout_MPR_sag', 'aa_scout_MPR_cor', 'aa_scout_MPR_tra', 'mprage_ND', 'mprage'
]
#now we can go into the folders and look for task folders in the session folders

for index, row in dicom_with_session_folders.iterrows():
    dicom_session_folder = row['dicom_index_folder_name']
    if dicom_session_folder is np.nan:
        print("the following item does not appear in the dicom index, although there may be a physical folder present:")
        print(row)
        continue
    print(". ", end="")
    try:
        #get the session folder raw list
        session_folder = srv.listdir(dicom_dir + dicom_session_folder)
        #xtract task names
        task_name_regex='Series_(\d+)_(.*)'

        run_names = [re.search(task_name_regex,x).group(2) for x in session_folder]
        #mark canonical sessions that are present in the run names
        present_sessions = [x for x in canonical_dicom_session_set if x in run_names]
        #mark canonical sessions missing from the run names
        missing_sessions = [x for x in canonical_dicom_session_set if x not in run_names]
        #mark other runs that are not canonical functional sessions
        other_sessions = [x for x in run_names if x not in canonical_dicom_session_set]
        #now save these in the dataframe
        dicom_with_session_folders.loc[index, 'dicom_present_sessions'] = ", ".join(present_sessions)
        dicom_with_session_folders.loc[index, 'dicom_missing_sessions'] = ", ".join(missing_sessions)
        dicom_with_session_folders.loc[index, 'dicom_other_sessions'] = ", ".join(other_sessions)
    except Exception as e:
        print("error with ", row)
        raise(e)
        


. . . . . . . . . . . . . . . . . . . . . . . . . . . . . the following item does not appear in the dicom index, although there may be a physical folder present:
dicom_folder_name           DEV018_20181025_190858
dicom_folder_name_sub_id                    DEV018
dicom_index_folder_name                        NaN
dicom_index_subject_id                         NaN
dicom_index_wave_id                            NaN
Name: 29, dtype: object
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . the following item does not appear in the dicom index, although there may be a physical folder present:
dicom_folder_name           DEV068_20181128_101017
dicom_folder_name_sub_id                    DEV068
dicom_index_folder_name                        NaN
dicom_index_subject_id                         NaN
dicom_index_wave_id                            NaN
Name: 124, d

### BIDS data

Let's get a list of the participants with BIDS folders

In [96]:
bids_folder_all = srv.listdir(bids_dir)

In [97]:
bids_folder_regex = 'sub-DEV\d\d\d'
bids_participants = [x for x in bids_folder_all if re.search(bids_folder_regex, x)]

In [99]:
bids_canonical_session_set= ['ROC1', 'ROC2', 'ROC3', 'ROC4', 'SST1', 'WTP1', 'WTP2', 'WTP3', 'WTP4','rest1','rest2']
#now get a list of bids sessions

#now get a list of fMRIPrep sessions
bids_session_df = pd.DataFrame(columns=['bids_subj_folder', 'bids_session_folder'])
for bids_subj_folder in bids_participants:
    print(". ", end="")
    
    bids_subj_path = bids_dir + '/' + bids_subj_folder
    bids_subj_dirlist = srv.listdir(bids_subj_path)
    bids_session_list = [x for x in bids_subj_dirlist if re.search('ses-wave\d+', x)]
    for bids_session_folder in bids_session_list:
        
        #now get the tasks that we have for each bids session
        bids_session_path = bids_subj_path + '/' + bids_session_folder + "/func/"
        nii_regex_code = bids_subj_folder + '_' + bids_session_folder + '_task-(\w+_acq-\d)(_run-\d+)?_bold.nii.gz'
        json_regex_code = bids_subj_folder + '_' + bids_session_folder + '_task-(\w+_acq-\d)(_run-\d+)?_bold.json'
        if srv.exists(bids_session_path):
            
            tasks_filename_list = srv.listdir(bids_session_path)
        else:
            tasks_filename_list = []
        

        nii_task_files = [re.search(nii_regex_code, x).group(1) for x in tasks_filename_list if re.search(nii_regex_code, x)]
        json_task_files = [re.search(json_regex_code, x).group(1) for x in tasks_filename_list if re.search(json_regex_code, x)]

        nii_task_abbrev_name = [s.replace("_acq-","") for s in nii_task_files]
        json_task_abbrev_name = [s.replace("_acq-","") for s in json_task_files]

        nii_sessions_present = [x for x in bids_canonical_session_set if x in nii_task_abbrev_name]
        nii_sessions_not_present = [x for x in bids_canonical_session_set if x not in nii_task_abbrev_name]
        nii_unrecognized_sessions = [x for x in nii_task_abbrev_name if x not in bids_canonical_session_set]

        json_sessions_present = [x for x in bids_canonical_session_set if x in json_task_abbrev_name]
        json_sessions_not_present = [x for x in bids_canonical_session_set if x not in json_task_abbrev_name]
        json_unrecognized_sessions = [x for x in json_task_abbrev_name if x not in bids_canonical_session_set]

        bids_session_df = pd.concat([bids_session_df,pd.DataFrame(
            {
                'bids_subj_folder': [bids_subj_folder], 
                'bids_session_folder': [bids_session_folder],
                #'nii_sessions_present': [nii_sessions_present],
                'nii_sessions_not_present': [", ".join(nii_sessions_not_present)],
                'nii_unrecognized_sessions': [", ".join(nii_unrecognized_sessions)],
                #'json_sessions_present': [json_sessions_present],
                'json_sessions_not_present': [", ".join(json_sessions_not_present)],
                'json_unrecognized_sessions': [", ".join(json_unrecognized_sessions)]
                })], ignore_index=True)


        #now check tehse againsg the canonical session set
    
    #print(fmriprep_subj_folder)

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

### fMRIPrep data

In [100]:

fMRIPrep_folder_all = srv.listdir(fmriprep_server_path)
fmriprep_folder_regex = 'sub-DEV\d\d\d$'
fmriprep_folder_list = [x for x in fMRIPrep_folder_all if re.search(fmriprep_folder_regex, x)]

#now get a list of fMRIPrep sessions
fmri_prep_session_df = pd.DataFrame(columns=['fmriprep_subj_folder', 'fmriprep_session_folder'])
for fmriprep_subj_folder in fmriprep_folder_list:
    print(". ", end="")
    fmriprep_subj_path = fmriprep_server_path + '/' + fmriprep_subj_folder
    fmriprep_subj_dirlist = srv.listdir(fmriprep_subj_path)
    fmri_prep_session_list = [x for x in fmriprep_subj_dirlist if re.search('ses-wave\d+', x)]
    for fmriprep_session_folder in fmri_prep_session_list:
        #let's find out what specific sessions are present and absent in fmriprep
        #smoothed data
        

        def check_for_data_present(regex_data_code):
            func_dir = fmriprep_subj_path + '/' + fmriprep_session_folder + '/func'
            if srv.exists(func_dir):
                session_func_files = srv.listdir(func_dir)
                data_present = [x for x in session_func_files if re.search(regex_data_code, x)]
            else:
                data_present = []
            #extract the task name
            data_present_task_name = [re.search(regex_data_code, x).group(1) for x in data_present]
            data_present_task_abbrev_name = [s.replace("_acq-","") for s in data_present_task_name]
            #print(data_present)
            #now mark which sessions are present and which are absent
            sessions_present = [x for x in canonical_session_set if x in data_present_task_abbrev_name]
            sessions_not_present = [x for x in canonical_session_set if x not in data_present_task_abbrev_name]
            unrecognized_sessions = [x for x in data_present_task_abbrev_name if x not in canonical_session_set]
            return(sessions_present, sessions_not_present, unrecognized_sessions)
        
        smoothed_regex_data_code = 's6_' + fmriprep_subj_folder + '_' + fmriprep_session_folder + '_task-([A-Z]*_acq-\d)_bold_space-MNI152NLin2009cAsym_preproc.nii'
        smoothed_sessions_present, smoothed_sessions_not_present, smoothed_unrecognized_sessions = check_for_data_present(smoothed_regex_data_code)

        regex_data_code = fmriprep_subj_folder + '_' + fmriprep_session_folder + '_task-([A-Z]*_acq-\d)_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'
        sessions_present, sessions_not_present, unrecognized_sessions = check_for_data_present(regex_data_code)


        #print(smoothed_data_present_task_abbrev_name)
        fmri_prep_session_df = pd.concat([fmri_prep_session_df,pd.DataFrame(
            {
                'fmriprep_subj_folder': [fmriprep_subj_folder], 
                'fmriprep_session_folder': [fmriprep_session_folder],
                'fmriprep_sessions_present': [", ".join(sessions_present)],
                'fmriprep_sessions_not_present': [", ".join(sessions_not_present)],
                'fmriprep_unrecognized_sessions': [", ".join(unrecognized_sessions)],
                'fmriprep_smoothed_sessions_present': [", ".join(smoothed_sessions_present)],
                'fmriprep_smoothed_sessions_not_present': [", ".join(smoothed_sessions_not_present)],
                'fmriprep_smoothed_unrecognized_sessions': [", ".join(smoothed_unrecognized_sessions)]
                })], ignore_index=True)


        

    #print(fmriprep_subj_folder)

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

## Redcap session list

In [101]:
redcap_sessions_list = pd.read_csv(redcap_csv_path)

In [102]:
redcap_sessions_list

,dev_id,redcap_event_name,date_0,date_1,date_2,date_3,date_4,date_5
0,DEV001,session_0_arm_1,2018-04-02,NaN,NaN,NaN,NaN,NaN
1,DEV001,session_1_arm_1,NaN,2018-04-06,NaN,NaN,NaN,NaN
2,DEV001,session_2_arm_1,NaN,NaN,2018-05-10,NaN,NaN,NaN
3,DEV001,session_3_arm_1,NaN,NaN,NaN,NaN,NaN,NaN
4,DEV004,session_0_arm_1,2018-04-11,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1459,DEV999,session_0_arm_1,NaN,NaN,NaN,NaN,NaN,NaN
1460,DEV999,session_1_arm_1,NaN,NaN,NaN,NaN,NaN,NaN
1461,TEST,session_0_arm_1,2019-10-19,NaN,NaN,NaN,NaN,NaN
1462,TEST,session_1_arm_1,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
#prepare the redcap list for matching
redcap_sessions_list['redcap_wave'] = redcap_sessions_list['redcap_event_name'].str.extract('session_(\d+)')[0]
#rename dev_id to redcap_dev_id
redcap_sessions_list.rename(columns={'dev_id': 'redcap_dev_id'}, inplace=True)
redcap_sessions_list['redcap_wave_int'] = redcap_sessions_list.redcap_wave.astype(int)


## Behavioral data

In [104]:


sst_behavioral_data_all = pd.read_csv(behavioral_data_sst_all_csv_filepath)
#merge redcap_sessions_list and sst_behavioral_data_all
#get counts of rows for each subject, wave, and run, in a new table
sst_behavioral_data_all_rowcount = sst_behavioral_data_all.groupby(['subid', 'waveid', 'runid']).size().reset_index(name='sst_behavioral_data_counts')
sst_behavioral_data_all_rowcount.waveid= sst_behavioral_data_all_rowcount.waveid.astype(int)

sr_behav_summary_df = pd.read_csv(self_report_behav_summary_data)

#redcap_sessions_list['redcap_wave'] = redcap_sessions_list.redcap_wave.astype(int)



# now merge sst_behavioral_data_all_rowcount and redcap_sessions_list
behavioral_data_index = pd.merge(
    redcap_sessions_list, sst_behavioral_data_all_rowcount, how='outer', left_on=['redcap_dev_id', 'redcap_wave_int'], right_on=['subid', 'waveid'])

    #now find rows where there was a redcap session but no behavioral data
behavioral_data_index.loc[(behavioral_data_index.sst_behavioral_data_counts.isnull()==True) & 
((behavioral_data_index.redcap_wave_int==1) | (behavioral_data_index.redcap_wave_int==2)),]

,redcap_dev_id,redcap_event_name,date_0,date_1,date_2,date_3,date_4,date_5,redcap_wave,redcap_wave_int,subid,waveid,runid,sst_behavioral_data_counts
1,DEV001,session_1_arm_1,NaN,2018-04-06,NaN,NaN,NaN,NaN,1,1.0,NaN,NaN,NaN,NaN
2,DEV001,session_2_arm_1,NaN,NaN,2018-05-10,NaN,NaN,NaN,2,2.0,NaN,NaN,NaN,NaN
159,DEV034,session_2_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,2,2.0,NaN,NaN,NaN,NaN
307,DEV061,session_1_arm_1,NaN,2019-01-11,NaN,NaN,NaN,NaN,1,1.0,NaN,NaN,NaN,NaN
308,DEV061,session_2_arm_1,NaN,NaN,2019-02-15,NaN,NaN,NaN,2,2.0,NaN,NaN,NaN,NaN
478,DEV092,session_1_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,NaN,NaN,NaN,NaN
504,DEV098,session_1_arm_1,NaN,2019-09-06,NaN,NaN,NaN,NaN,1,1.0,NaN,NaN,NaN,NaN
705,DEV136,session_1_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,NaN,NaN,NaN,NaN
737,DEV142,session_1_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,NaN,NaN,NaN,NaN
757,DEV146,session_1_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,NaN,NaN,NaN,NaN


In [18]:
wtp_behavioral_data_all = pd.read_csv(behavioral_data_wtp_all_csv_filepath)

In [21]:
wtp_behavioral_data_all

,Unnamed: 0,event_id,isi_pre,onset,duration,food_pic,food_num,cond,health_cond,liking_cond,liking_rating,response,isi_post,end,run,wave,subject
0,0,1,3.000000,6.070811,6.555327,PeanutButterCup.bmp,1,unhealthy_liked,unhealthy,liked,3,NaN,3.000000,12.626138,run1,1,DEV089
1,1,2,3.000000,18.669634,6.522478,Bananas.bmp,2,healthy_liked,healthy,liked,4,7.0,1.780283,25.192112,run1,1,DEV089
2,2,3,1.780283,30.008762,6.527172,RedApple.bmp,3,healthy_liked,healthy,liked,3,7.0,0.095117,36.535933,run1,1,DEV089
3,3,4,0.095117,39.674699,6.528184,100grand_small.bmp,4,unhealthy_disliked,unhealthy,disliked,1,5.0,2.680966,46.202883,run1,1,DEV089
4,4,5,2.680966,51.936189,6.548751,PotatoChips.bmp,5,unhealthy_liked,unhealthy,liked,3,5.0,0.319097,58.484941,run1,1,DEV089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70123,11,12,0.925946,127.108305,6.532239,MacNCheese.bmp,12,unhealthy_disliked,unhealthy,disliked,3,7.0,0.670865,133.640544,run4,2,DEV234
70124,12,13,0.670865,137.357405,6.531946,AppleChips.bmp,13,healthy_disliked,healthy,disliked,1,5.0,0.859947,143.889352,run4,2,DEV234
70125,13,14,0.859947,147.788792,6.532961,PeasPlease.bmp,14,healthy_disliked,healthy,disliked,1,6.0,0.333113,154.321753,run4,2,DEV234
70126,14,15,0.333113,157.692138,6.515675,Strawberries.bmp,15,healthy_liked,healthy,liked,4,8.0,1.421652,164.207813,run4,2,DEV234


In [29]:
#group by subject, wave, and run, and get counts of rows
wtp_behavioral_data_all_rowcount = wtp_behavioral_data_all.groupby(['subject', 'wave', 'run']).size().reset_index(name='wtp_behavioral_data_counts')
wtp_behavioral_data_all_rowcount.groupby(['wave','run']).wtp_behavioral_data_counts.value_counts()

wave  run   wtp_behavioral_data_counts
1     run1  16                            287
      run2  16                            282
      run3  16                            282
      run4  16                            282
2     run1  16                            255
      run2  16                            255
      run3  16                            255
      run4  16                            253
3     run1  32                            235
      run2  32                            231
4     run1  32                            192
      run2  32                            191
5     run1  32                            134
      run2  32                            133
Name: wtp_behavioral_data_counts, dtype: int64

In [31]:
#create an aggregate list of runs present for each wave in a new table grouped by subject and wave
wtp_behavioral_data_all_rowcount_agg = wtp_behavioral_data_all_rowcount.groupby(['subject', 'wave']).wtp_behavioral_data_counts.agg(['sum', 'count']).reset_index()

In [35]:
wtp_behavioral_data_all_rowcount_agg.columns=['subject','wave','wtp_trialcount','wtp_runcount']

In [34]:
wtp_behavioral_data_all_rowcount_agg

,subject,wave,trialcount,runcount
0,DEV001,1,64,4
1,DEV002,1,64,4
2,DEV004,1,64,4
3,DEV004,2,64,4
4,DEV004,3,64,2
...,...,...,...,...
1098,DEV998,1,16,1
1099,DEV999,1,64,4
1100,DEV999,3,32,1
1101,DEV999,4,64,2


## Matching

In [105]:

# prepare the bids list for matching
bids_session_df['bids_wave'] = bids_session_df['bids_session_folder'].str.extract('ses-wave(\d+)')[0]
bids_session_df['bids_dev_id'] = bids_session_df['bids_subj_folder'].str.extract('sub-(DEV\d+)')[0]

#prepare the fmriprep list for matching

fmri_prep_session_df['fmriprep_dev_id'] = fmri_prep_session_df['fmriprep_subj_folder'].str.extract('sub-(DEV\d\d\d)')[0]
fmri_prep_session_df['fmriprep_wave'] = fmri_prep_session_df['fmriprep_session_folder'].str.extract('ses-wave(\d+)')[0]


In [106]:
dicom_with_session_folders

,dicom_folder_name,dicom_folder_name_sub_id,dicom_index_folder_name,dicom_index_subject_id,dicom_index_wave_id,dicom_present_sessions,dicom_missing_sessions,dicom_other_sessions
0,DEV001_20180406_170438,DEV001,DEV001_20180406_170438,DEV001,1,"ROC1, ROC2, ROC3, ROC4, SST1, WTP1, WTP2, WTP3...","Resting_1_PhysioLog, Resting_2_PhysioLog",
1,DEV001_20180510_162136,DEV001,DEV001_20180510_162136,DEV001,2,"ROC1, ROC2, ROC3, ROC4, WTP1, WTP2, WTP3, WTP4...","SST1, Resting_1_PhysioLog, Resting_2_PhysioLog...",
2,DEV004_20180416_183517,DEV004,DEV004_20180416_183517,DEV004,1,"ROC1, ROC2, ROC3, ROC4, SST1, WTP1, WTP2, WTP3...","Resting_2, Resting_1_PhysioLog, Resting_2_Phys...",Flair_axial.sw
3,DEV004_20180515_180522,DEV004,DEV004_20180515_180522,DEV004,2,"ROC1, ROC2, ROC3, ROC4, SST1, WTP1, WTP2, WTP3...","Resting_1_PhysioLog, Resting_2_PhysioLog",
4,DEV005_20180425_185327,DEV005,DEV005_20180425_185327,DEV005,1,"ROC1, ROC2, ROC3, ROC4, SST1, WTP1, WTP2, WTP3...","Resting_1_PhysioLog, Resting_2_PhysioLog",
...,...,...,...,...,...,...,...,...
544,DEV312_20220831_130818,DEV312,DEV312_20220831_130818,DEV312,2,"ROC1, ROC2, ROC3, ROC4, SST1, WTP1, WTP2, WTP3...",,
545,dev125_20190823_102411,dev125,dev125_20190823_102411,DEV125,2,"ROC1, ROC2, ROC3, ROC4, SST1, WTP1, WTP2, WTP3...",,
546,dev133_20190808_171200,dev133,dev133_20190808_171200,DEV133,1,"ROC1, ROC2, ROC3, ROC4, SST1, WTP1, WTP2, WTP3...",,
547,NaN,NaN,DEV_20210608_170401,DEV198,3,"ROC1, ROC2, ROC3, ROC4, SST1, WTP1, WTP2, WTP3...","Resting_2, Resting_2_PhysioLog",


In [107]:
redcap_sessions_list.redcap_wave
fmri_prep_session_df.fmriprep_wave

0      1
1      2
2      1
3      2
4      1
      ..
539    2
540    1
541    2
542    1
543    2
Name: fmriprep_wave, Length: 544, dtype: object

In [108]:
matched_list_1 = pd.merge(redcap_sessions_list,fmri_prep_session_df, how='outer', 
    left_on=['redcap_dev_id', 'redcap_wave'],
    right_on=['fmriprep_dev_id', 'fmriprep_wave'])
matched_list_2 = pd.merge(matched_list_1,bids_session_df, how='outer',
    left_on=['redcap_dev_id', 'redcap_wave'],
    right_on=['bids_dev_id', 'bids_wave'])
matched_list_3 = pd.merge(matched_list_2,dicom_with_session_folders, how='outer',
    left_on=['redcap_dev_id', 'redcap_wave'],
    right_on=['dicom_index_subject_id', 'dicom_index_wave_id'])
matched_list_4 = pd.merge(matched_list_3,participants_included, how='outer',
    left_on=['redcap_dev_id'],
    right_on=['mastersheet_dev_id'])


matched_list_5 = pd.merge(
    matched_list_4, sst_behavioral_data_all_rowcount, how='outer', 
    left_on=['redcap_dev_id', 'redcap_wave_int'], 
    right_on=['subid', 'waveid'])
matched_list_6 = pd.merge(
    matched_list_5, wtp_behavioral_data_all_rowcount_agg, how='outer',
    left_on=['redcap_dev_id', 'redcap_wave_int'],
    right_on=['subject', 'wave'])


In [109]:
matched_list_full = matched_list_5

In [110]:
matched_list_full

,redcap_dev_id,redcap_event_name,date_0,date_1,date_2,date_3,date_4,date_5,redcap_wave,redcap_wave_int,...,dicom_index_wave_id,dicom_present_sessions,dicom_missing_sessions,dicom_other_sessions,mastersheet_ppt_number,mastersheet_dev_id,subid,waveid,runid,sst_behavioral_data_counts
0,DEV001,session_0_arm_1,2018-04-02,NaN,NaN,NaN,NaN,NaN,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DEV001,session_1_arm_1,NaN,2018-04-06,NaN,NaN,NaN,NaN,1,1.0,...,1,"ROC1, ROC2, ROC3, ROC4, SST1, WTP1, WTP2, WTP3...","Resting_1_PhysioLog, Resting_2_PhysioLog",,NaN,NaN,NaN,NaN,NaN,NaN
2,DEV001,session_2_arm_1,NaN,NaN,2018-05-10,NaN,NaN,NaN,2,2.0,...,2,"ROC1, ROC2, ROC3, ROC4, WTP1, WTP2, WTP3, WTP4...","SST1, Resting_1_PhysioLog, Resting_2_PhysioLog...",,NaN,NaN,NaN,NaN,NaN,NaN
3,DEV001,session_3_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,3,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DEV004,session_0_arm_1,2018-04-11,NaN,NaN,NaN,NaN,NaN,0,0.0,...,NaN,NaN,NaN,NaN,1.0,DEV004,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,DEV309,3.0,1.0,256.0
1520,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,DEV310,3.0,1.0,256.0
1521,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,DEV311,3.0,1.0,256.0
1522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,DEV312,3.0,1.0,256.0


In [111]:
matched_list_full.to_csv(output_data_path + 'match_out.csv')

## Mark missing data

In [112]:
matched_list_full['fmriprep_present'] = matched_list_full['fmriprep_session_folder'].isnull()==False
matched_list_full['bids_present'] = matched_list_full['bids_session_folder'].isnull()==False
matched_list_full['bids present, fmriprep not present'] = (matched_list_full['bids_present'] == True) & (matched_list_full['fmriprep_present'] == False)
matched_list_full['dicom_present'] = matched_list_full['dicom_index_subject_id'].isnull()==False
matched_list_full['dicom present, bids not present']= (matched_list_full['dicom_present'] == True) & (matched_list_full['bids_present'] == False)

## Cross-reference DEV QC and Exclusions with Redcap to get a new list of DEV subjects

### get the data from QC Exclusions

In [113]:
#access a google sheet from python
from gsheets import Sheets
sheets = Sheets.from_files('~/.ssh/client_secret.json', '~/.ssh/storage.json')

In [114]:
qc_exclusions = sheets.get('https://docs.google.com/spreadsheets/d/1NywNwiRxwlAOjb2eZ7q58MklRWUGvgE5hlaILurwFIs/edit#gid=0')

In [115]:
wave_1 = qc_exclusions.sheets[0].to_frame()
wave_2 = qc_exclusions.sheets[1].to_frame()

In [116]:
wave_2

,subjectID,dicom_present,Unnamed: 2,bids_present,"dicom present, bids not present",fmriprep_present,"bids present, fmriprep not present",reason missing,DICOM session missing explanations,dicom_present_sessions,...,WTP1 motion,WTP2 SDC,WTP2 reg,WTP2 motion,WTP3 SDC,WTP3 reg,WTP3 motion,WTP4 SDC,WTP4 reg,WTP4 motion
0,DEV001,True,NaN,True,False,True,False,NaN,"""error with SST""","ROC1, ROC2, ROC3, ROC4, WTP1, WTP2, WTP3, WTP4...",...,0.0,missing,x,0.0,missing,x,1.0,missing,x,1.0
1,DEV004,True,NaN,True,False,True,False,NaN,NaN,"ROC1, ROC2, ROC3, ROC4, SST1, WTP1, WTP2, WTP3...",...,0.0,x,x,0.0,x,x,0.0,x,x,0.0
2,DEV005,True,NaN,True,False,True,False,NaN,NaN,"ROC1, ROC2, ROC3, ROC4, SST1, WTP1, WTP2, WTP3...",...,0.0,x,x,0.0,x,x,0.0,x,x,0.0
3,DEV006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DEV007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,DEV308,True,NaN,True,False,True,False,NaN,NaN,"ROC1, ROC2, ROC3, ROC4, SST1, WTP1, WTP2, WTP3...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267,DEV309,True,NaN,True,False,True,False,NaN,NaN,"ROC1, ROC2, ROC3, ROC4, SST1, WTP1, WTP2, WTP3...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
268,DEV310,True,NaN,True,False,True,False,NaN,NaN,"ROC1, ROC2, ROC3, ROC4, SST1, WTP1, WTP2, WTP3...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
269,DEV311,True,NaN,True,False,True,False,NaN,NaN,"ROC1, ROC2, ROC3, ROC4, SST1, WTP1, WTP2, WTP3...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Add new Wave1 subjects from redcap

In [117]:
w_id=1
QC_preexisting_subjs = wave_1['subjectID']
print('Wave ' + str(w_id) + ' subjects to add to spreadsheet')
redcap_dev_id_wave_i=redcap_sessions_list.loc[redcap_sessions_list.redcap_wave==str(w_id),'redcap_dev_id']
subjects_to_add_to_spreadsheet = redcap_dev_id_wave_i.loc[~redcap_dev_id_wave_i.isin(QC_preexisting_subjs)]
print(subjects_to_add_to_spreadsheet)
#null_output =[print(s) for s in subjects_to_add_to_spreadsheet]

Wave 1 subjects to add to spreadsheet
Series([], Name: redcap_dev_id, dtype: object)


In [118]:
w_id=2
QC_preexisting_subjs = wave_2['subjectID']
print('Wave ' + str(w_id) + ' subjects to add to spreadsheet')
redcap_dev_id_wave_i=redcap_sessions_list.loc[redcap_sessions_list.redcap_wave==str(w_id),'redcap_dev_id']
subjects_to_add_to_spreadsheet = redcap_dev_id_wave_i.loc[~redcap_dev_id_wave_i.isin(QC_preexisting_subjs)]
print(subjects_to_add_to_spreadsheet)
#null_output =[print(s) for s in subjects_to_add_to_spreadsheet]

Wave 2 subjects to add to spreadsheet
1463    TEST
Name: redcap_dev_id, dtype: object


I manually added these to the google sheet. Now, we can start to update why these various values are missing.

### use the QC Exclusions list as an index to order the merged data above and create data frames taht can be manually imported back into the sheet

In [119]:
wave_2['subjectID']
#matched_list_full_w2['redcap_dev_id']

0      DEV001
1      DEV004
2      DEV005
3      DEV006
4      DEV007
        ...  
266    DEV308
267    DEV309
268    DEV310
269    DEV311
270    DEV312
Name: subjectID, Length: 271, dtype: object

In [120]:
matched_list_full_w1 = matched_list_full.loc[matched_list_full.redcap_wave_int==1,:]
QC_master_ordered_match_list_w1 = pd.merge(wave_1['subjectID'], matched_list_full_w1, how='left', left_on='subjectID', right_on='redcap_dev_id')
QC_master_ordered_match_list_w1.to_csv(output_data_path + '/QC_master_ordered_match_list_w1.csv')

In [121]:
matched_list_full_w2 = matched_list_full.loc[matched_list_full.redcap_wave_int==2,:]
QC_master_ordered_match_list_w2 = pd.merge(wave_2['subjectID'], matched_list_full_w2, how='left', left_on='subjectID', right_on='redcap_dev_id')
QC_master_ordered_match_list_w2.to_csv(output_data_path + '/QC_master_ordered_match_list_w2.csv')

In [122]:
matched_list_full.loc[matched_list_full.redcap_wave=='2',:]

,redcap_dev_id,redcap_event_name,date_0,date_1,date_2,date_3,date_4,date_5,redcap_wave,redcap_wave_int,...,mastersheet_dev_id,subid,waveid,runid,sst_behavioral_data_counts,fmriprep_present,bids_present,"bids present, fmriprep not present",dicom_present,"dicom present, bids not present"
2,DEV001,session_2_arm_1,NaN,NaN,2018-05-10,NaN,NaN,NaN,2,2.0,...,NaN,NaN,NaN,NaN,NaN,True,True,False,True,False
6,DEV004,session_2_arm_1,NaN,NaN,2018-05-15,NaN,NaN,NaN,2,2.0,...,DEV004,DEV004,2.0,1.0,256.0,True,True,False,True,False
12,DEV005,session_2_arm_1,NaN,NaN,2018-06-12,NaN,NaN,NaN,2,2.0,...,NaN,DEV005,2.0,1.0,256.0,True,True,False,True,False
18,DEV008,session_2_arm_1,NaN,NaN,2018-06-11,NaN,NaN,NaN,2,2.0,...,NaN,DEV008,2.0,1.0,256.0,True,True,False,True,False
21,DEV009,session_2_arm_1,NaN,NaN,2018-06-05,NaN,NaN,NaN,2,2.0,...,DEV009,DEV009,2.0,1.0,256.0,True,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445,DEV309,session_2_arm_1,NaN,NaN,2022-09-13,NaN,NaN,NaN,2,2.0,...,DEV309,DEV309,2.0,1.0,256.0,True,True,False,True,False
1448,DEV310,session_2_arm_1,NaN,NaN,2022-09-11,NaN,NaN,NaN,2,2.0,...,DEV310,DEV310,2.0,1.0,256.0,True,True,False,True,False
1451,DEV311,session_2_arm_1,NaN,NaN,2022-09-16,NaN,NaN,NaN,2,2.0,...,DEV311,DEV311,2.0,1.0,256.0,True,True,False,True,False
1454,DEV312,session_2_arm_1,NaN,NaN,2022-08-31,NaN,NaN,NaN,2,2.0,...,DEV312,DEV312,2.0,1.0,256.0,True,True,False,True,False
